# Proširenje

In [1]:
# Imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Loading data
df_assessments = pd.read_csv('assessments.csv')
df_courses = pd.read_csv('courses.csv')
df_studentAssessment = pd.read_csv('studentAssessment.csv')
df_studentInfo = pd.read_csv('studentInfo.csv')
df_studentRegistration = pd.read_csv('studentRegistration.csv')
df_studentVle = pd.read_csv('studentVle.csv')
df_vle = pd.read_csv('vle.csv')

# Dropout prediction

## Preprocessing

In [3]:
df_dropout = df_studentInfo[['code_module', 'code_presentation', 'id_student', 'final_result']]
df_dropout['final_result'].replace({'Pass':0, 'Fail':0, 'Distinction':0, 'Withdrawn':1}, inplace=True)
# DF sa svim student-predmet kombinacijama i njihov final result; 1->dropout, 0->not dropout
df_dropout

C:\Users\Vito\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,code_module,code_presentation,id_student,final_result
0,AAA,2013J,11391,0
1,AAA,2013J,28400,0
2,AAA,2013J,30268,1
3,AAA,2013J,31604,0
4,AAA,2013J,32885,0
...,...,...,...,...
32588,GGG,2014J,2640965,0
32589,GGG,2014J,2645731,0
32590,GGG,2014J,2648187,0
32591,GGG,2014J,2679821,1


In [4]:
print("Dropout rate is "+str(round(len(df_dropout[df_dropout['final_result'] == 1])/len(df_dropout), 5)))
print("Our model needs to have accuracy > "+str(1-0.3116))

Dropout rate is 0.3116
Our model needs to have accuracy > 0.6884


In [5]:
df_stud_demographic = pd.get_dummies(df_studentInfo, columns=['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability'], drop_first=True)
df_stud_demographic['final_result'].replace({'Pass':0, 'Fail':0, 'Distinction':0, 'Withdrawn':1}, inplace=True)
# DF sa demografskin znacajkama studenata
df_stud_demographic

,code_module,code_presentation,id_student,num_of_prev_attempts,studied_credits,final_result,gender_M,region_East Midlands Region,region_Ireland,region_London Region,...,imd_band_30-40%,imd_band_40-50%,imd_band_50-60%,imd_band_60-70%,imd_band_70-80%,imd_band_80-90%,imd_band_90-100%,age_band_35-55,age_band_55<=,disability_Y
0,AAA,2013J,11391,0,240,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,AAA,2013J,28400,0,60,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,AAA,2013J,30268,0,60,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
3,AAA,2013J,31604,0,60,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,AAA,2013J,32885,0,60,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,0,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32589,GGG,2014J,2645731,0,30,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
32590,GGG,2014J,2648187,0,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32591,GGG,2014J,2679821,0,30,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [6]:
df_stud_Assessment = df_studentAssessment.merge(df_assessments.drop(['date', 'weight'], axis=1), how='left', on=['id_assessment'])
df_stud_Assessment = df_stud_Assessment[df_stud_Assessment['assessment_type'] != 'Exam']
df_stud_Assessment = pd.get_dummies(df_stud_Assessment, columns=['assessment_type'], drop_first=True).drop(['is_banked'], axis=1)
df_stud_Assessment = df_stud_Assessment.drop(['id_assessment', 'date_submitted'], axis=1).groupby(['id_student', 'code_module', 'code_presentation', 'assessment_type_TMA']).mean().reset_index()
df_stud_Assessment = df_stud_Assessment.merge(df_dropout, how='left', on=['code_module', 'code_presentation', 'id_student'])
# DF sa avg score svakog studenta za dvije razlicite vrse assessmenta
df_stud_Assessment

,id_student,code_module,code_presentation,assessment_type_TMA,score,final_result
0,6516,AAA,2014J,1,61.800000,0
1,8462,DDD,2013J,1,87.666667,1
2,8462,DDD,2014J,1,86.500000,1
3,11391,AAA,2013J,1,82.000000,0
4,23629,BBB,2013B,0,100.000000,0
...,...,...,...,...,...,...
39915,2698535,CCC,2014B,0,24.666667,1
39916,2698535,CCC,2014B,1,23.000000,1
39917,2698535,EEE,2013J,1,54.250000,0
39918,2698577,BBB,2014J,1,64.400000,0


In [7]:
df_activity_sum = df_studentVle.drop(['date'], axis=1).merge(df_vle.drop(['week_from', 'week_to'], axis=1), how='left', on=['id_site', 'code_module', 'code_presentation'])
df_activity_sum = df_activity_sum.drop(['id_site'], axis=1)
df_activity_sum = df_activity_sum.groupby(['code_module', 'code_presentation', 'id_student', 'activity_type']).sum()
df_activity_sum = df_activity_sum.reset_index()
df_activity_sum = df_activity_sum.set_index(['code_module', 'code_presentation', 'id_student', 'activity_type']).unstack(['activity_type'])
df_activity_sum.columns = ['{}_{}'.format(t, v) for v, t in df_activity_sum.columns]
df_activity_sum = df_activity_sum.reset_index()
df_activity_sum = df_activity_sum.fillna(0)
df_activity_sum = df_activity_sum.merge(df_dropout, how='left', on=['code_module', 'code_presentation', 'id_student'])
# DF sa sumom klikova po aktivnostima za svakog studenta i predmete
df_activity_sum

,code_module,code_presentation,id_student,dataplus_sum_click,dualpane_sum_click,externalquiz_sum_click,folder_sum_click,forumng_sum_click,glossary_sum_click,homepage_sum_click,...,ouwiki_sum_click,page_sum_click,questionnaire_sum_click,quiz_sum_click,repeatactivity_sum_click,resource_sum_click,sharedsubpage_sum_click,subpage_sum_click,url_sum_click,final_result
0,AAA,2013J,11391,0.0,0.0,0.0,0.0,193.0,0.0,138.0,...,0.0,0.0,0.0,0.0,0.0,13.0,0.0,32.0,5.0,0
1,AAA,2013J,28400,10.0,0.0,0.0,0.0,417.0,0.0,324.0,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,87.0,48.0,0
2,AAA,2013J,30268,0.0,0.0,0.0,0.0,126.0,0.0,59.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,22.0,4.0,1
3,AAA,2013J,31604,2.0,0.0,0.0,0.0,634.0,1.0,432.0,...,0.0,0.0,0.0,0.0,0.0,19.0,0.0,144.0,90.0,0
4,AAA,2013J,32885,0.0,0.0,0.0,0.0,194.0,4.0,204.0,...,0.0,0.0,0.0,0.0,0.0,45.0,0.0,79.0,14.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,GGG,2014J,2640965,0.0,0.0,0.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0
29224,GGG,2014J,2645731,0.0,0.0,0.0,0.0,65.0,5.0,167.0,...,0.0,0.0,0.0,152.0,0.0,109.0,0.0,47.0,0.0,0
29225,GGG,2014J,2648187,0.0,0.0,0.0,0.0,0.0,1.0,63.0,...,0.0,0.0,0.0,130.0,0.0,19.0,0.0,20.0,0.0,0
29226,GGG,2014J,2679821,0.0,0.0,0.0,0.0,118.0,0.0,65.0,...,0.0,0.0,0.0,31.0,0.0,9.0,0.0,12.0,0.0,1


In [8]:
df_activity_num = df_studentVle.drop(['date'], axis=1).merge(df_vle.drop(['week_from', 'week_to'], axis=1), how='left', on=['id_site', 'code_module', 'code_presentation'])
df_activity_num = df_activity_num.drop(['id_site'], axis=1)
df_activity_num = df_activity_num.groupby(['code_module', 'code_presentation', 'id_student', 'activity_type']).count()
df_activity_num = df_activity_num.reset_index()
df_activity_num = df_activity_num.rename(columns={'sum_click':'num_click'})
df_activity_num = df_activity_num.set_index(['code_module', 'code_presentation', 'id_student', 'activity_type']).unstack(['activity_type'])
df_activity_num.columns = ['{}_{}'.format(t, v) for v, t in df_activity_num.columns]
df_activity_num = df_activity_num.reset_index()
df_activity_num = df_activity_num.fillna(0)
df_activity_num = df_activity_num.merge(df_dropout, how='left', on=['code_module', 'code_presentation', 'id_student'])
# DF sa brojem pristupa aktivnostima po aktivnostima za svakog studenta i predmete
df_activity_num

,code_module,code_presentation,id_student,dataplus_num_click,dualpane_num_click,externalquiz_num_click,folder_num_click,forumng_num_click,glossary_num_click,homepage_num_click,...,ouwiki_num_click,page_num_click,questionnaire_num_click,quiz_num_click,repeatactivity_num_click,resource_num_click,sharedsubpage_num_click,subpage_num_click,url_num_click,final_result
0,AAA,2013J,11391,0.0,0.0,0.0,0.0,52.0,0.0,40.0,...,0.0,0.0,0.0,0.0,0.0,11.0,0.0,11.0,4.0,0
1,AAA,2013J,28400,3.0,0.0,0.0,0.0,163.0,0.0,80.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,42.0,27.0,0
2,AAA,2013J,30268,0.0,0.0,0.0,0.0,30.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,12.0,4.0,1
3,AAA,2013J,31604,1.0,0.0,0.0,0.0,197.0,1.0,121.0,...,0.0,0.0,0.0,0.0,0.0,13.0,0.0,83.0,51.0,0
4,AAA,2013J,32885,0.0,0.0,0.0,0.0,77.0,3.0,68.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,53.0,13.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29223,GGG,2014J,2640965,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,6.0,0.0,0
29224,GGG,2014J,2645731,0.0,0.0,0.0,0.0,14.0,2.0,35.0,...,0.0,0.0,0.0,38.0,0.0,67.0,0.0,20.0,0.0,0
29225,GGG,2014J,2648187,0.0,0.0,0.0,0.0,0.0,1.0,22.0,...,0.0,0.0,0.0,36.0,0.0,14.0,0.0,14.0,0.0,0
29226,GGG,2014J,2679821,0.0,0.0,0.0,0.0,17.0,0.0,13.0,...,0.0,0.0,0.0,5.0,0.0,9.0,0.0,6.0,0.0,1


## Implementation

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [ ]:
num_RFC = []
num_RFC_minmax = []
num_RFC_standard = []
num_GBM = []
num_GBM_minmax = []
num_GBM_standard = []
num_LR = []
num_LR_minmax = []
num_LR_standard = []
X_num = df_activity_num[['dataplus_num_click','dualpane_num_click','externalquiz_num_click','folder_num_click','forumng_num_click','glossary_num_click','homepage_num_click','htmlactivity_num_click','oucollaborate_num_click','oucontent_num_click','ouelluminate_num_click','ouwiki_num_click','page_num_click','questionnaire_num_click','quiz_num_click','repeatactivity_num_click','resource_num_click','sharedsubpage_num_click','subpage_num_click','url_num_click']].values
y_num = df_activity_num['final_result'].values

kf = KFold(n_splits=10)
kf.get_n_splits(X_num)
counter = 0
for train_index, test_index in kf.split(X_num):
    counter+= 1
    print(str(counter)+" Fold")
    scaler1 = MinMaxScaler()
    scaler2 = StandardScaler()

    X_train, X_test = X_num[train_index], X_num[test_index]
    y_train, y_test = y_num[train_index], y_num[test_index]
    X_train_minmax = scaler1.fit_transform(X_train)
    X_test_minmax = scaler1.transform(X_test)
    X_train_standard = scaler2.fit_transform(X_train)
    X_test_standard = scaler2.transform(X_test)
    
    clf1 = RandomForestClassifier()
    clf2 = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1, max_depth=3, random_state=1)
    clf3 = LogisticRegression(max_iter=1000)
    clf1.fit(X_train, y_train)
    clf2.fit(X_train, y_train)
    clf3.fit(X_train, y_train)
    num_RFC.append(clf1.score(X_test, y_test))
    num_GBM.append(clf2.score(X_test, y_test))
    num_LR.append(clf3.score(X_test, y_test))
    
    clf1 = RandomForestClassifier()
    clf2 = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1, max_depth=3, random_state=1)
    clf3 = LogisticRegression(max_iter=1000)
    clf1.fit(X_train_minmax, y_train)
    clf2.fit(X_train_minmax, y_train)
    clf3.fit(X_train_minmax, y_train)
    num_RFC_minmax.append(clf1.score(X_test_minmax, y_test))
    num_GBM_minmax.append(clf2.score(X_test_minmax, y_test))
    num_LR_minmax.append(clf3.score(X_test_minmax, y_test))
    
    clf1 = RandomForestClassifier()
    clf2 = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1, max_depth=3, random_state=1)
    clf3 = LogisticRegression(max_iter=1000)
    clf1.fit(X_train_standard, y_train)
    clf2.fit(X_train_standard, y_train)
    clf3.fit(X_train_standard, y_train)
    num_RFC_standard.append(clf1.score(X_test_standard, y_test))
    num_GBM_standard.append(clf2.score(X_test_standard, y_test))
    num_LR_standard.append(clf3.score(X_test_standard, y_test))

print("Avg 10-fold accuracy of RFC "+str(np.mean(np.array(num_RFC))))
print("Avg 10-fold accuracy of GBM "+str(np.mean(np.array(num_GBM))))
print("Avg 10-fold accuracy of LR "+str(np.mean(np.array(num_LR))))

print("Avg 10-fold accuracy of RFC minmax "+str(np.mean(np.array(num_RFC_minmax))))
print("Avg 10-fold accuracy of GBM minmax "+str(np.mean(np.array(num_GBM_minmax))))
print("Avg 10-fold accuracy of LR minmax "+str(np.mean(np.array(num_LR_minmax))))

print("Avg 10-fold accuracy of RFC standard "+str(np.mean(np.array(num_RFC_standard))))
print("Avg 10-fold accuracy of GBM standard "+str(np.mean(np.array(num_GBM_standard))))
print("Avg 10-fold accuracy of LR  standard "+str(np.mean(np.array(num_LR_standard))))

1 Fold
2 Fold


c:\users\vito\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3 Fold


## Prediction of success based on demographics and previous education

In [ ]:
## imd_band, region, highest_education ----> pomocu decision tree
## ali probaj i logisticka regresija, SVM, NaiveBayes i random forest

## Prediction of success based on gender